In [3]:
import numpy as np
from scipy import signal,interpolate
import time

In [2]:
##Augmentations from quipu

def noiseAug(xs, std = 0.1):#Noise augmentation
    return xs + np.random.normal(0, std, xs.shape)

def magnitudeAug(xs, std = 0.1):#Magnitude augmentation
    return xs * np.abs(np.random.normal(1, std, len(xs)).reshape((-1,1)) ) 

def stretchAug(xs, std = 0.1, probability = 0.5,std_fill=0.08):#Stretch augmentation
    x_new = np.copy(xs)
    sample_len=len(x_new[1])
    for i in range(len(xs)):
        if np.random.rand() > 1-probability:
            x_resampled=signal.resample(x_new[i], int(sample_len*np.random.normal(1, std))) 
            if len(x_resampled)<700:
                x_resampled= np.append(x_resampled,np.random.normal(0, std_fill, sample_len - len(x_resampled)))
            else:
                x_resampled=x_resampled[:700]
            x_new[i] = x_resampled;
    return x_new

In [12]:
##Augmentations with cupy
import cupy as cp
import cupyx

def noiseAug_gpu(xs, std = 0.1):#Noise augmentation
    return xs + cp.random.normal(0, std, xs.shape)

def magnitudeAug_gpu(xs, std = 0.1):
    return xs * cp.abs(cp.random.normal(1, std, len(xs)).reshape((-1,1)) ) 

def stretchAug_gpu(xs, std = 0.1,std_fill=0.01, probability = 0.5,):
    x_new = np.copy(xs)
    sample_len=len(x_new[1])
    for i in range(len(xs)):
        if cp.random.rand() > 1-probability:
            x_resampled=cupyx.scipy.signal.resample(x_new[i], int(sample_len*cp.random.normal(1, std))) 
            if len(x_resampled)<700:
                x_resampled= cp.append(x_resampled,cp.random.normal(0, std_fill, sample_len - len(x_resampled)))
            else:
                x_resampled=x_resampled[:700]
            x_new[i] = x_resampled;
    return x_new

In [4]:
trace_to_use_example=np.arange(700); ##Just a trace to 
xs=np.tile(trace_to_use_example, (60000, 1));
xs_gpu=cp.array(xs)
def timeAndTestFnc(xs,fnc, nameFnc=""):
    start = time.time()
    fnc(xs)
    end = time.time()
    print("Time of "+ nameFnc + ": "+ str(end - start))

In [13]:
timeAndTestFnc(xs,noiseAug, nameFnc="CPU Noise Aug")
timeAndTestFnc(xs,magnitudeAug, nameFnc="CPU Mag Aug")
timeAndTestFnc(xs,stretchAug, nameFnc="CPU Stretch Aug")

timeAndTestFnc(xs_gpu,noiseAug_gpu, nameFnc="GPU Noise Aug")
timeAndTestFnc(xs_gpu,magnitudeAug_gpu, nameFnc="GPU Mag Aug")
timeAndTestFnc(xs_gpu,stretchAug_gpu, nameFnc="GPU Stretch Aug")

Time of CPU Noise Aug: 1.0989360809326172
Time of CPU Mag Aug: 0.11512517929077148
Time of CPU Stretch Aug: 2.1354572772979736
Time of GPU Noise Aug: 0.0013997554779052734
Time of GPU Mag Aug: 0.00010967254638671875


AttributeError: module 'cupyx.scipy.signal' has no attribute 'resample'

In [34]:
cupyx.scipy.interpolate

AttributeError: module 'cupyx.scipy' has no attribute 'interpolate'